In [11]:
using Pkg; Pkg.activate("/home/jovyan/NFMOneNotebook")
using GenericRofReader
using Plots
using Logging
using Statistics
Logging.disable_logging(Logging.Warn)

  Activating project at `~/NFMOneNotebook`


LogLevel(1001)

In [12]:
# Define the struct FMDStatusMessage containing the status of FMD for all four cameras
struct ViperFMDStatusMessage
    fmd_status_cam_rear::Vector{Int32}
    fmd_status_cam_left::Vector{Int32}
    fmd_status_cam_front::Vector{Int32}
    fmd_status_cam_right::Vector{Int32}

    #constructor to create empty instance of the struct
    function ViperFMDStatusMessage()
        new(Vector{Int32}(),
            Vector{Int32}(),
            Vector{Int32}(),
            Vector{Int32}()
        )
    end
end

function print_fmd_message(fmdMsg::ViperFMDStatusMessage)
        println("fmd_status_cam_rear ...")
        println(fmdMsg.fmd_status_cam_rear)

        println("fmd_status_cam_left ...")
        println(fmdMsg.fmd_status_cam_left)

        println("fmd_status_cam_front ...")
        println(fmdMsg.fmd_status_cam_front)

        println("fmd_status_cam_right ...")
        println(fmdMsg.fmd_status_cam_right)    
end

function parse_viper_fmd_status(interface_samples)::ViperFMDStatusMessage
    parsed_data = ViperFMDStatusMessage()

    try
        for element in interface_samples
            push!(parsed_data.fmd_status_cam_rear, element.m_status_cam_rear)
            push!(parsed_data.fmd_status_cam_left, element.m_status_cam_left)
            push!(parsed_data.fmd_status_cam_front, element.m_status_cam_front)
            push!(parsed_data.fmd_status_cam_right, element.m_status_cam_right)
        end
    catch e
        println("exception: $e")
        @error("An error occurred in parsing viper fmd status data: $e", "exit")
    end
    return parsed_data
end

parse_viper_fmd_status (generic function with 1 method)

In [24]:
using OneParkingReaders

#using OneParkingReaders.General: get_interface_reader_generic
using OneParkingReaders.Utils: get_interface_reader_generic

function extract_FMDstatus(rof_folder_path)::ViperFMDStatusMessage
    
    interface_name = "FMDStatusMessage.c2w__fast_misalignment_status.FMDStatusMessage"

    println("Interface name: $interface_name")

    fullpath = joinpath(rof_folder_path, interface_name)

    reader = get_interface_reader_generic(fullpath; create_enums = false)
    
    all_valid_interface_samples = read_all_valid_interfaces(reader)
    
    print("length of all_valid_interface_samples: ") 
    println(length(all_valid_interface_samples))

    fmd_status_message = parse_viper_fmd_status(all_valid_interface_samples)

    return fmd_status_message
end

extract_FMDstatus (generic function with 1 method)

In [14]:
@enum CameraId begin
    camera_rear
    camera_left
    camera_front
    camera_right
end

# Initialize the KPIs
kpis = Dict{String, Union{Missing, Float64}}(
    "fmd_status_good_rate_rear" => missing,
    "fmd_status_good_rate_left" => missing,
    "fmd_status_good_rate_front" => missing,
    "fmd_status_good_rate_right" => missing
)

# Initialize the FMD Status
fmd_status = Dict{String, Union{Missing, String}}(
    "rear" => missing,
    "left" => missing,
    "front" => missing,
    "right" => missing
)

# Initialize the FMD Status
fmd_status_val = Dict{String, Union{Missing, Int32}}(
    "rear" => missing,
    "left" => missing,
    "front" => missing,
    "right" => missing
)

Dict{String, Union{Missing, Int32}} with 4 entries:
  "left"  => missing
  "right" => missing
  "rear"  => missing
  "front" => missing

In [21]:
function fmd_status_good_rate_calculator!(kpis, cam_str, fmd_status_message, expected_fmd_status)
    # Calculates the good rate percentage of the fmd status going backwards
    
    fmd_status_cam = getfield(fmd_status_message, Symbol("fmd_status_cam_", cam_str))
    expected_fmd_status_cam = expected_fmd_status[cam_str]

    # Initialize the index to the last element of the vector and count to zero
    index = length(fmd_status_cam)
    count_good_frames_from_end::Int = 0

    fmd_status_val[cam_str] = last(fmd_status_cam)

    while(index > 0)
        if(fmd_status_cam[index] != expected_fmd_status_cam)
            break
        end
        count_good_frames_from_end += 1
        index -= 1
    end

    fmd_status_good_rate::Float64 =
        count_good_frames_from_end / length(fmd_status_cam) * 100

    fmd_status_good_rate = round(fmd_status_good_rate, digits=2)

    if (count_good_frames_from_end == 0)
        # All angles of the camera are NOT within the acceptable range for all frames
        @warn("camera: $cam_str, fmd_status_good_rate: $(fmd_status_good_rate) % ,
         all angles of the camera are NOT within the acceptable range at last frame")
    end

    println("fmd_status_good_rate: $(fmd_status_good_rate)")

    kpis["fmd_status_good_rate_" * cam_str] = fmd_status_good_rate
    
end

fmd_status_good_rate_calculator! (generic function with 1 method)

In [22]:
#Define the variants of badly mounted vehicle
bad_case_vehicle_list = ["USB09", "USB13", "DEB06"]

function extract_vehicle_variant(file_path::String)
    # Split the file path by underscores
    parts = split(file_path, '_')
    
    if length(parts) >= 2
        return parts[2]
    else
        return "No vechile match found in $(file_path)"
    end
end

function isOutlierSequence(vehicle_variant::String, string_list::Vector{String})::Bool
    # Check if the target_string is in the string_list
    return vehicle_variant in bad_case_vehicle_list
end

function updateExpectedFMDStatus!(expected_fmd_status, vehicle_variant::String)

    # This sequence has atleast one of the camera badly mounted
    if(vehicle_variant == "DEB06")
            # deadjusted : At least one angle of the angles is outside the acceptable range
            # we expect the fmd status to be deadjusted, i.e value 3
            expected_fmd_status["rear"]  = 4
            expected_fmd_status["left"]  = 4
            expected_fmd_status["front"] = 3
            expected_fmd_status["right"] = 4
    
    elseif(vehicle_variant == "USB09")
            # deadjusted : At least one angle of the angles is outside the acceptable range
            # we expect the fmd status to be deadjusted, i.e value 3
            expected_fmd_status["rear"]  = 4
            expected_fmd_status["left"]  = 3
            expected_fmd_status["front"] = 4
            expected_fmd_status["right"] = 4

    elseif(vehicle_variant == "USB13")
            # deadjusted : At least one angle of the angles is outside the acceptable range
            # we expect the fmd status to be deadjusted, i.e value 3
            expected_fmd_status["rear"]  = 3
            expected_fmd_status["left"]  = 3
            expected_fmd_status["front"] = 4
            expected_fmd_status["right"] = 3
    end

end

updateExpectedFMDStatus! (generic function with 1 method)

In [17]:
using CSV
using DataFrames

function append_row(file_path::String, row_data::Any)

    df = DataFrame(row_data)
    
    if isfile(file_path)
        # Append mode: Write without header
        CSV.write(file_path, df, append=true, header=false)
    else
        # Create new file or overwrite existing file
        CSV.write(file_path, df)
    end
end

append_row (generic function with 1 method)

In [18]:
function isGoodORBad(good_rate_cam)
    if(good_rate_cam > 0.0)
        return "Correct"
    else 
        return "InCorrect"
    end
end

function areAllGoodORBad(kpis)
    if ( kpis["fmd_status_good_rate_rear"] > 0 &&  kpis["fmd_status_good_rate_left"] > 0 && kpis["fmd_status_good_rate_front"] > 0  && kpis["fmd_status_good_rate_right"] > 0)
        return "Correct"
    else 
        return "InCorrect"
    end
end

areAllGoodORBad (generic function with 1 method)

In [27]:
using Glob

#hash_foc_run = "5949953b4aa70f36221f5ed345e7fb9c9c71cfff796cec4d04e7a91becfd9979"

hash_foc_run = "8f6d50a824751f2ccbdf970d0e91cacdc6046fd5d1412a1a4350b38281fbcace"  #USB13
#hash_foc_run = "e97701cfb82df2cf8c895d4d1254f785161e109c20e797f93250fe0c54a94ba6"   #DEB06
#hash_foc_run = "16dafef3f75518ec02795c8e49b0174736a298eed579a5c18aea9355954f1edd"   #DEB05

rof_base_dir = joinpath("../data/ReadOnly/dyperdrsprod/ford-dat-3", hash_foc_run, "recompute_output")

# good : All angles of the camera are within the acceptable range
expected_fmd_status = Dict("rear" => 4, "left" => 4, "front" => 4, "right" => 4)

# Define the CSV file path
csv_file_path = "kpis_fmd_status.csv"

function calculate_fmd_status(base_dir::String)
    # Define the patterns
    patterns = ["*.mf4/*.rof"]

    # Loop through the patterns and find matching files
    for pattern in patterns
        # Use glob to find files matching the pattern in the base directory
        for rof_folder_path in glob(joinpath(base_dir, pattern))
            
            println("rof_folder_path ........................................................................................... ")
            println(rof_folder_path)

            # Split the string by '/'
            parts = split(rof_folder_path, '/')
            
            # Extract the hash and file path
            hash_value = parts[6]       # The hash value
            # Extract the part of the file path before the extension
            file_path_with_extension = split(parts[8], '.')[1]
            file_path::String = file_path_with_extension
            
            println("Seqeunce: $file_path")

            vehicle_variant::String = extract_vehicle_variant(file_path)

            println("Vehicle variant: $vehicle_variant")

            if(isOutlierSequence(vehicle_variant, bad_case_vehicle_list))
                updateExpectedFMDStatus!(expected_fmd_status, vehicle_variant)
            end

            fmd_status_message = extract_FMDstatus(rof_folder_path)

            for cam_id in instances(CameraId)
        
                # Get camera string.
                cam_str::AbstractString = split(string(cam_id), "_", limit = 2)[2]
    
                fmd_status_good_rate_calculator!(kpis, cam_str, fmd_status_message, expected_fmd_status)

                good_rate_cam = kpis["fmd_status_good_rate_" * cam_str ]

                fmd_status[cam_str] = isGoodORBad(good_rate_cam)

                println("fmd_status[$cam_str]: $(fmd_status[cam_str]) ")
       
            end

            status_all_cam = areAllGoodORBad(kpis)
                   
            row = Dict(
                    :Hash => hash_value, 
                    :Sequence => file_path,
                    :FMD_status_rear => fmd_status["rear"],
                    :FMD_status_left => fmd_status["left"], 
                    :FMD_status_front => fmd_status["front"],   
                    :FMD_status_right => fmd_status["right"], 
                    :FMD_status_all_cam => status_all_cam,
                    :FMD_status_last_rear => fmd_status_val["rear"],
                    :FMD_status_last_left => fmd_status_val["left"], 
                    :FMD_status_last_front => fmd_status_val["front"],   
                    :FMD_status_last_right => fmd_status_val["right"], 
                    :FMD_status_good_rate_rear => kpis["fmd_status_good_rate_rear"] ,
                    :FMD_status_good_rate_left =>  kpis["fmd_status_good_rate_left"],
                    :FMD_status_good_rate_front =>  kpis["fmd_status_good_rate_front"],
                    :FMD_status_good_rate_right =>  kpis["fmd_status_good_rate_right"] 
            )
                append_row(csv_file_path, row)

        end
    end
end

calculate_fmd_status(rof_base_dir)

rof_folder_path ........................................................................................... 
../data/ReadOnly/dyperdrsprod/ford-dat-3/8f6d50a824751f2ccbdf970d0e91cacdc6046fd5d1412a1a4350b38281fbcace/recompute_output/FD3_USB13_20240717_190239_001.mf4/recall_FD3_USB13_20240717_190239_001.rof
Seqeunce: FD3_USB13_20240717_190239_001
Vehicle variant: USB13
Interface name: FMDStatusMessage.c2w__fast_misalignment_status.FMDStatusMessage
length of all_valid_interface_samples: 1116
fmd_status_good_rate: 86.65
fmd_status[rear]: Correct 
fmd_status_good_rate: 86.65
fmd_status[left]: Correct 
fmd_status_good_rate: 0.0
fmd_status[front]: InCorrect 
fmd_status_good_rate: 86.65
fmd_status[right]: Correct 
rof_folder_path ........................................................................................... 
../data/ReadOnly/dyperdrsprod/ford-dat-3/8f6d50a824751f2ccbdf970d0e91cacdc6046fd5d1412a1a4350b38281fbcace/recompute_output/FD3_USB13_20240717_190511_001.mf4/recall_FD3_USB13_2

Hash Value: 5949953b4aa70f36221f5ed345e7fb9c9c71cfff796cec4d04e7a91becfd9979
File Path: FD3_USB13_20240717_194407_001
